In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results
import os

In [ ]:
root = Path("/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/sweep_results")
sweep_name = "sweep01_neutralization_v2"
fig_path = root / "figures"
os.makedirs(fig_path, exist_ok=True)
# load data frames from sweep
params_df = pd.read_csv(root / f"{sweep_name}_params_df.csv")
nodal_df = pd.read_csv(root / f"{sweep_name}_nodal_df.csv")
lefty_df = pd.read_csv(root / f"{sweep_name}_lefty_df.csv")
rho_df = pd.read_csv(root / f"{sweep_name}_rho_df.csv")

nodal_df.head()

### Calculate Nodal and Lefty section scores

In [ ]:
import ast  # safer than eval for Python literals

dx = 10
L = 3000

# target domain sigma: 250
d_sigma = 250
ref_grid = np.linspace(0, L, int(L/dx)) - L/2

# generate calculation masks
mid_mask = np.abs(ref_grid) <= d_sigma
left_mask = ref_grid < -d_sigma
right_mask = ref_grid > d_sigma

# generate lefty and nodal arrays
lefty_array = lefty_df.iloc[:, 1].values
lefty_array= np.array([np.array(ast.literal_eval(s), dtype=float) for s in lefty_array])
nodal_array = nodal_df.iloc[:, 1].values
nodal_array= np.array([np.array(ast.literal_eval(s), dtype=float) for s in nodal_array])

In [ ]:
# calculate values
nodal_factor_vec = np.asarray(np.divide(params_df["sigma_N"], params_df["mu_L"]*1.11e-4/0.61e-4,))
lefty_factor_vec = np.asarray(np.divide(params_df["sigma_L"], params_df["mu_L"]))
nodal_array_norm = np.divide(nodal_array, nodal_factor_vec[:, None])
lefty_array_norm = np.divide(lefty_array, lefty_factor_vec[:, None])
params_df["N_factor"] = nodal_factor_vec
params_df["L_factor"] = lefty_factor_vec

L_left = np.mean(lefty_array_norm[:, left_mask==1], axis=1)
L_center = np.mean(lefty_array_norm[:, mid_mask==1], axis=1)
L_right = np.mean(lefty_array_norm[:, right_mask==1], axis=1)

N_left = np.mean(nodal_array_norm[:, left_mask==1], axis=1)
N_center = np.mean(nodal_array_norm[:, mid_mask==1], axis=1)
N_right = np.mean(nodal_array_norm[:, right_mask==1], axis=1)

# L_left = np.mean(lefty_array[:, left_mask==1], axis=1)
# L_center = np.mean(lefty_array[:, mid_mask==1], axis=1)
# L_right = np.mean(lefty_array[:, right_mask==1], axis=1)

# N_left = np.mean(nodal_array[:, left_mask==1], axis=1)
# N_center = np.mean(nodal_array[:, mid_mask==1], axis=1)
# N_right = np.mean(nodal_array[:, right_mask==1], axis=1)

In [ ]:
params_df["N_left"] = N_left
params_df["N_center"] = N_center 
params_df["N_right"] = N_right

params_df["L_left"] = L_left
params_df["L_center"] = L_center 
params_df["L_right"] = L_right

In [ ]:
fig = px.scatter(params_df, x="N_left", y="N_center", color="N_amp", hover_data={"sim_id", "mu_L"})
# add a line at y=1
# fig.add_hline(y=1, line_dash="dash", line_color="white", annotation_text="y=1", annotation_position="top left")
# # add vertical line at x=500/3500
# fig.add_vline(x=500/3500, line_dash="dash", line_color="white", annotation_text="N0", annotation_position="top right")
# # set axis limits
fig.update_xaxes(title_text="[Nodal] (periphery)")
fig.update_yaxes(title_text="[Nodal] (center)")

fig.show()

In [ ]:
fig = px.scatter(params_df, x="L_center", y="N_center", color="L_factor", hover_data={"sim_id", "mu_L"})
# add a line at y=1
# fig.add_hline(y=1, line_dash="dash", line_color="white", annotation_text="y=1", annotation_position="top left")
# # add vertical line at x=500/3500
# fig.add_vline(x=500/3500, line_dash="dash", line_color="white", annotation_text="N0", annotation_position="top right")
# # set axis limits
fig.update_xaxes(title_text="[Lefty] (center)")
fig.update_yaxes(title_text="[Nodal] (center)")

fig.show()

In [ ]:
from src.utilities.plot_functions import format_2d_plotly

sim_id = "c4bdd83d"
plot_filter = params_df["sim_id"]==sim_id
nodal_vec = nodal_array[plot_filter][0]
lefty_vec = lefty_array[plot_filter][0]

fig = go.Figure()
fig.add_traces(go.Scatter(
    x=ref_grid, 
    y=nodal_vec , 
    mode="lines", 
    line=dict(width=3, color="#8da0cb"),  # Set2 blue
    fill="tozeroy",
    fillcolor="rgba(141, 160, 203, 0.6)",  # semi-transparent blue
    name="Nodal"
))
fig.add_traces(go.Scatter(
    x=ref_grid, 
    y=lefty_vec, 
    mode="lines", 
    line=dict(width=3, color="#fc8d62"),  # Set2 red
    fill="tozeroy",
    fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
    name="Lefty"
))

# fig.update_xaxes(title_text="[Nodal] (periphery)")
# fig.update_yaxes(title_text="[Nodal] (center)", range=[0, 6])
fig = format_2d_plotly(fig, axis_labels=["position (microns)", "concentration (nmol per micron squared)"], 
                       font_size=20)

fig.show()

In [ ]:
# params_df.loc[plot_filter, :].iloc[0].to_dict()

## Run simulation

In [ ]:
from symmetry_breaking.models.NLP_neutralization_1D import NodalLeftyNeutralization1D
from symmetry_breaking.models.trackers import NodalROITracker
from symmetry_breaking.models.sweep import run_simulation_1D, make_1d_grid
from pde import ProgressTracker, PlotTracker

# hyperparams
dx = 5
L = 3500
T = 10 * 60 * 60
dt = 0.5 * dx ** 2 / 60 / 1.25 # Stability condition for diffusio

param_dict = {
         'K_NL': 138.9495494373,
         'K_A': 193.0697728883,
         'K_R': 19.3069772888,
         'K_I': 19.3069772888,
         'mu_L': 0.0002275846,
         'N_amp': 517.9474679231,
         'N_sigma': 31.6227766017,
         'sigma_N': 10.0,
         'sigma_L': 10.0,
         'D0_N': 1.85,
         'D0_L': 15.0,
         'alpha_L': 0,
         'alpha_N': 0,
         'tau_rho': 3600,
         'n': 2,
         'm': 1,
         'p': 2,
         'q': 2}

static_params = {
                  "sigma_N": 10.0,  # Nodal auto-activation
                  "no_density_dependence": True,
                  "alpha_L": 0,
                  "alpha_N": 0,
                  "tau_rho": 3600,
                }

sim_config = {
                "dx": dx,
                "L": L,
                "T": T,
                "dt": dt,
                "model_class": NodalLeftyNeutralization1D,
                "tracker_class": NodalROITracker,
                "interval": 1000,
            }

sim_inputs = (param_dict | static_params, sim_config, "") 

In [ ]:
param_dict, sim_config, output_dir = sim_inputs

# Unpack sim config
dx = sim_config["dx"]
L = sim_config["L"]
T = sim_config["T"]
dt = sim_config["dt"]
interval=300
model_class = sim_config["model_class"]
tracker_class = sim_config["tracker_class"]
# interval = sim_config.get("interval", 1000)

# --- Setup grid and model ---
grid = make_1d_grid(length=L, dx=dx)
model = model_class(**param_dict)
state = model.get_state(grid)

# roi = NodalROITracker(grid, interval=interval,
#                       save_profiles=False,
#                       store_every=300,      # set e.g. 100 if you want snapshots too
#                       downsample=None,          # optional
#                       dtype=np.float32)

# --- Build a tracker collection ---
# Progress bar updates every 'interval' steps
progress = ProgressTracker(interval=interval)

# Live plots (you can adjust scale, cmap, etc.)
# plots = PlotTracker(interval=interval, plot_args={"figsize": (6, 4)}, 
#                     show=True, tight_layout=True)

# If you want both at once, wrap them in a list
trackers = [progress, tracker_class(grid, interval=interval)]

# --- Run simulation ---
state = model.solve(state, t_range=T, dt=dt, tracker=trackers)

# --- Collect results from your custom tracker ---
result = {
    **param_dict,
    **trackers[-1].get_metrics(),  # last tracker is your custom one
}

In [ ]:
profiles = trackers[-1].get_profiles()
x = profiles["x"]- 1750
time_vec = profiles["times"]
N = profiles["Activator"]      # numpy array
L = profiles["Repressor"]      # numpy array
rho = profiles.get("rho") 

In [ ]:
from tqdm import tqdm

frame_path = fig_path / f"{sim_id}_frames"
os.makedirs(frame_path, exist_ok=True)

for t, time in enumerate(tqdm(time_vec)):
    fig = go.Figure()
    fig.add_traces(go.Scatter(
        x=x, 
        y=N[t, :] , 
        mode="lines", 
        line=dict(width=3, color="#8da0cb"),  # Set2 blue
        fill="tozeroy",
        fillcolor="rgba(141, 160, 203, 0.6)",  # semi-transparent blue
        name="Nodal"
    ))
    fig.add_traces(go.Scatter(
        x=x, 
        y=L[t, :], 
        mode="lines", 
        line=dict(width=3, color="#fc8d62"),  # Set2 red
        fill="tozeroy",
        fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
        name="Lefty"
    ))
    
    fig = format_2d_plotly(fig, axis_labels=["position (microns)", "concentration (nmol per micron squared)"], 
                           font_size=18)
    fig.update_layout(title=f"Nodal and Lefty concentrations ({np.round(time/3600,2)} hours)")
    fig.update_xaxes(range=[-1500, 1500])
    fig.update_yaxes(range=[0, 20000])
    
    # fig.show()
    fig.write_image(frame_path / f"frame_{t:05}.png")

fig.show()

### Re-run the exact same simulation but with no link between Lefty and Nodal 

In [ ]:

# hyperparams
dx = 5
L = 3500
T = 10 * 60 * 60
dt = 0.5 * dx ** 2 / 60 / 1.25 # Stability condition for diffusio

param_dict = {
         'K_NL': 138.9495494373,
         'K_A': 193.0697728883,
         # 'K_R': 19.3069772888,
         'K_I': 1e10, #19.3069772888,
         'mu_L': 0.0002275846,
         'N_amp': 517.9474679231,
         'N_sigma': 31.6227766017,
         'sigma_N': 10.0,
         'sigma_L': 10.0,
         'D0_N': 1.85,
         'D0_L': 15.0,
         'alpha_L': 0,
         'alpha_N': 0,
         'tau_rho': 3600,
         'n': 2,
         'm': 1,
         'p': 2,
         'q': 2}

static_params = {
                  "sigma_N": 10.0,  # Nodal auto-activation
                  "no_density_dependence": True,
                  "alpha_L": 0,
                  "alpha_N": 0,
                  "tau_rho": 3600,
                }

sim_config = {
                "dx": dx,
                "L": L,
                "T": T,
                "dt": dt,
                "model_class": NodalLeftyNeutralization1D,
                "tracker_class": NodalROITracker,
                "interval": 1000,
            }

sim_inputs = (param_dict | static_params, sim_config, "") 

In [ ]:
25**2

In [ ]:
param_dict, sim_config, output_dir = sim_inputs

# Unpack sim config
dx = sim_config["dx"]
L = sim_config["L"]
T = sim_config["T"]
dt = sim_config["dt"]
interval=300
model_class = sim_config["model_class"]
tracker_class = sim_config["tracker_class"]
# interval = sim_config.get("interval", 1000)

# --- Setup grid and model ---
grid = make_1d_grid(length=L, dx=dx)
model = model_class(**param_dict)
state = model.get_state(grid)

# roi = NodalROITracker(grid, interval=interval,
#                       save_profiles=False,
#                       store_every=300,      # set e.g. 100 if you want snapshots too
#                       downsample=None,          # optional
#                       dtype=np.float32)

# --- Build a tracker collection ---
# Progress bar updates every 'interval' steps
progress = ProgressTracker(interval=interval)

# Live plots (you can adjust scale, cmap, etc.)
# plots = PlotTracker(interval=interval, plot_args={"figsize": (6, 4)}, 
#                     show=True, tight_layout=True)

# If you want both at once, wrap them in a list
trackers = [progress, tracker_class(grid, interval=interval)]

# --- Run simulation ---
state = model.solve(state, t_range=T, dt=dt, tracker=trackers)

# --- Collect results from your custom tracker ---
result = {
    **param_dict,
    **trackers[-1].get_metrics(),  # last tracker is your custom one
}

In [ ]:
profiles = trackers[-1].get_profiles()
x = profiles["x"]- 1750
time_vec = profiles["times"]
N = profiles["Activator"]      # numpy array
L = profiles["Repressor"]      # numpy array
rho = profiles.get("rho") 

In [ ]:
frame_path = fig_path / f"{sim_id}_frames_no_LN"
os.makedirs(frame_path, exist_ok=True)

for t, time in enumerate(tqdm(time_vec)):
    fig = go.Figure()
    fig.add_traces(go.Scatter(
        x=x, 
        y=N[t, :] , 
        mode="lines", 
        line=dict(width=3, color="#8da0cb"),  # Set2 blue
        fill="tozeroy",
        fillcolor="rgba(141, 160, 203, 0.8)",  # semi-transparent blue
        name="Nodal"
    ))
    fig.add_traces(go.Scatter(
        x=x, 
        y=L[t, :], 
        mode="lines", 
        line=dict(width=3, color="#fc8d62"),  # Set2 red
        fill="tozeroy",
        fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
        name="Lefty"
    ))
    
    fig = format_2d_plotly(fig, axis_labels=["position (microns)", "concentration (nmol per micron squared)"], 
                           font_size=18)
    fig.update_layout(title=f"Nodal and Lefty concentrations ({np.round(time/3600,2)} hours)")
    fig.update_xaxes(range=[-1500, 1500])
    fig.update_yaxes(range=[0, 50000])
    
    # fig.show()
    fig.write_image(frame_path / f"frame_{t:05}.png")

fig.show()

In [ ]:
# fig.update_yaxes(range=[0, 60000])
# fig.show()